### 본 매뉴얼은 데이터 분석에 필요한 데이터셋을 빈티지별로 저장하고 관리하는 방법에 대해 다룬다.

# 실시간 인플레이션 전망을 위한 빈티지 데이터셋 생성
직원 누구나 BIDAS 데이터 분석 자원(backend.AI, fasttrack, s3 등)를 이용해 데이터셋 생성 코드를 매일/매주/매월 배치 실행하여, 자동으로 빈티지 데이터셋을 저장하고 관리할 수 있다. 아래 코드는 디지털신기술팀에서 운영 중인 실시간 인플레이션 전망 모형에 필요한 빈티지 데이터셋을 생성하는 예시 코드이다. 
- 작성자: 디지털신기술팀 이창훈 과장

데이터셋에 대한 자세한 내용은 BOK 이슈노트 `[제2024-5호] 빅데이터와 기계학습 알고리즘을 활용한 실시간 인플레이션 전망(real-time inflation forecasting)`[[💾이슈노트 다운로드]](https://bidas-s3.boknet.intra/newtech/public/inf_nowcasting/이슈노트2024-5호.pdf) 참고.

In [1]:
# 필요한 라이브러리 불러오기
import bok_da as bd
import pickle

import pandas as pd
from pandas.tseries.offsets import QuarterEnd, QuarterBegin, MonthEnd, MonthBegin, BDay
from pandas.tseries.offsets import DateOffset

import requests
import math
import joblib

import numpy as np
from numpy import asarray

from datetime import datetime
from multiprocessing import Process

import glob
import os

from time import time
from tqdm import tqdm
import itertools

idx = pd.IndexSlice

### 빈티지셋 날짜 설정

In [2]:
b_date = '2000-1'

c_date = datetime.now()
c_year = c_date.year
c_month = c_date.month
e_date = f'{c_year}-{c_month}'

데이터셋 생성에 필요한 메타데이터 저장 경로 설정

In [3]:
s3_repo_path = "s3://newtech/public/inf_nowcasting/input"

In [4]:
vspec = pd.read_excel(f'{s3_repo_path}/data_list_all_v8.xlsx', index_col = None)
vspec.index.names = [None]

vspec_import = vspec.loc[vspec['Data ID'] != 'calculated']
vspec

,Group,Series Name,cat,Acode,Tcode1,Adcode,Gcode,My ID,Data ID
0,소비자물가,"지출목적별 소비자물가지수(품목포함, 2020＝100) (월,분기,년 1965.01~...",1,w1lag1,2,0.0,1.0,P_cpi_1,KOSIS-101_DT_1J22001-M-T-T10-0-14101_ausgabe
1,NaN,"지출목적별 소비자물가지수(품목포함, 2020＝100) (월,분기,년 1965.01~...",1,w1lag1,2,0.0,2.0,P_cpi_2,KOSIS-101_DT_1J22001-M-T-T10-A-14101_ausgabe
2,NaN,"지출목적별 소비자물가지수(품목포함, 2020＝100) (월,분기,년 1965.01~...",1,w1lag1,2,0.0,2.0,P_cpi_3,KOSIS-101_DT_1J22001-M-T-T10-C-14101_ausgabe
3,NaN,"지출목적별 소비자물가지수(품목포함, 2020＝100) (월,분기,년 1965.01~...",1,w1lag1,2,0.0,2.0,P_cpi_4,KOSIS-101_DT_1J22001-M-T-T10-D-14101_ausgabe
4,NaN,"지출목적별 소비자물가지수(품목포함, 2020＝100) (월,분기,년 1965.01~...",1,w1lag1,2,0.0,2.0,P_cpi_5,KOSIS-101_DT_1J22001-M-T-T10-E-14101_ausgabe
...,...,...,...,...,...,...,...,...,...
317,NaN,통합재정수지 (월 1999.12~2023.05)-월-재정수지-자본지출-십억원,3,w3lag2,2,0.0,3.0,F_fb_5,KOSIS-102_DT_102N_AD01-M-16102AD1-15102AD123-1...
318,NaN,통합재정수지 (월 1999.12~2023.05)-월-재정수지-통합재정수지-십억원,1,w3lag2,3,0.0,3.0,F_fb_6,KOSIS-102_DT_102N_AD01-M-16102AD1-15102AD130-1...
319,텍스트,뉴스심리지수(실험적 통계)-월-뉴스심리지수,1,w1lag1,1,1.0,2.0,T_nsi,NECOS-521U001-M-A001
320,NaN,US Economic Policy Uncertainty Composite Index,1,w2lag1,1,1.0,1.0,T_usepu,BBG-BBG-EPUCCUSM_Index


In [5]:
today = datetime.today()
vintage_date = today.strftime('%Y-%m-%d')

In [6]:
groups = ['P', 'PB', 'L', 'C', 'LM', 'IE', 'EI', 'RE', 'EG', 'S', 'GB', 'F', 'T']

In [7]:
adm_vspec = pd.read_excel(f'{s3_repo_path}/AdmPrices.xlsx', sheet_name='AdmPrices_list', index_col = None)
adm_vspec.index.names = [None]
adm_var_list = [var for var in adm_vspec['My ID']]
dfa = bd.get_bidas_data(adm_vspec.loc[adm_vspec['My ID'].isin(adm_var_list), 'Data ID'], adm_var_list).loc[b_date:e_date].astype(float).resample('M').mean()

### 변수 그룹별 데이터 불러온 후 전처리하여 데이터프레임(DF, DF_raw) 생성

In [8]:
DF = pd.DataFrame()
DF_raw = pd.DataFrame()

for group in groups:
    
    var_list = [var for var in vspec_import['My ID'] if var.split('_')[0] == group]
    df = bd.get_bidas_data(vspec.loc[vspec['My ID'].isin(var_list), 'Data ID'], var_list).loc[b_date:e_date].astype(float).resample('M').mean()
    DF_raw = pd.concat([DF_raw, df], axis=1)
    
    if group == 'IE':
        df['IE_mir_8'] = df['IE_mir_5'] - df['IE_mir_2']
        
    elif group == 'P':
        adm_vspec = pd.read_excel(f'{s3_repo_path}/AdmPrices.xlsx', sheet_name='AdmPrices_list', index_col = None)
        adm_vspec.index.names = [None]
        adm_var_list = [var for var in adm_vspec['My ID']]
        dfa = bd.get_bidas_data(adm_vspec.loc[adm_vspec['My ID'].isin(adm_var_list), 'Data ID'], adm_var_list).loc[b_date:e_date].astype(float).resample('M').mean()
        P_a, P_ea = bd.gen_P_adm_eadm_v2(dfa, adm_vspec.weights, b_date, e_date)
        
        df['P_adm'] = P_a
        df['P_eadm'] = P_ea
        
    elif group == 'L':
        df.loc[:, 'L_wh_1'] = df.loc[:, ['L_wh_1_a', 'L_wh_1_b', 'L_wh_1_c']].mean(axis=1)
        df.loc[:, 'L_wh_2'] = df.loc[:, ['L_wh_2_a', 'L_wh_2_b', 'L_wh_2_c']].mean(axis=1)
        df.loc[:, 'L_wh_3'] = df.loc[:, ['L_wh_3_a', 'L_wh_3_b', 'L_wh_3_c']].mean(axis=1)
        df.loc[:, 'L_tw_1'] = df.loc[:, ['L_tw_1_a', 'L_tw_1_b', 'L_tw_1_c']].mean(axis=1)
        df.loc[:, 'L_tw_2'] = df.loc[:, ['L_tw_2_a', 'L_tw_2_b', 'L_tw_2_c']].mean(axis=1)
        df.loc[:, 'L_tw_3'] = df.loc[:, ['L_tw_3_a', 'L_tw_3_b', 'L_tw_3_c']].mean(axis=1)
        df.loc[:, 'L_vjp'] = df.loc[:, ['L_vjp_a', 'L_vjp_b']].mean(axis=1)
        df['L_vu'] = df['L_vjp'] / df['L_eap_4'] # 실업자대비 빈일자리 비율

        # drop columns L_wh_1_a, L_wh_1_b, ..., L_tw_3_c
        for p in ['L_' + p + '_' + q + '_' + r for p in ['wh', 'tw'] for q in ['1', '2', '3'] for r in ['a', 'b', 'c']]:
            df = df.drop(p, axis=1)
        df = df.drop(['L_vjp_a', 'L_vjp_b'], axis=1)            
        
    elif group == 'RE':
        df.loc[:, 'RE_ccs_1'] = df.loc[:, ['RE_ccs_1_a', 'RE_ccs_1_b']].mean(axis=1)
        df.loc[:, 'RE_ccs_2'] = df.loc[:, ['RE_ccs_2_a', 'RE_ccs_2_b']].mean(axis=1)

        # drop columns RE_ccs_2_a ...
        df = df.drop(['RE_ccs_1_a', 'RE_ccs_1_b', 'RE_ccs_2_a', 'RE_ccs_2_b'], axis=1)  
        
    else:
        pass

    
    for var in df.columns:
        
        dfi = df[var]
        
        if vspec.loc[vspec['My ID'].eq(var), 'Tcode1'].values[0] == 2:
            if (dfi <= 0).any().any():
                dfi = dfi.pct_change(12)*100
            elif var == 'P_cpi_1': # Nan 인 경우 ffill 되어 마지막 값 nan인 경우 이상한 값이 채워짐
                dfi = dfi.dropna().pct_change(12)*100
            else:
                dfi = (np.log(dfi) - np.log(dfi.shift(12)))*100
                
        elif vspec.loc[vspec['My ID'].eq(var), 'Tcode1'].values[0] == 3:
            dfi = dfi - dfi.shift(1)
            
        DF = pd.concat([DF, dfi], axis=1)

DF.index = pd.to_datetime(DF.index)
DF_raw.index = pd.to_datetime(DF_raw.index)

## P_ppi_2, 생산자물가지수(기본분류)-상품 2020년으로 기준년 개편되면서 삭제

P_ppi_2 is not imported.


In [9]:
DF

,P_cpi_1,P_cpi_2,P_cpi_3,P_cpi_4,P_cpi_5,P_cpi_6,P_cpi_7,P_cpi_8,P_cpi_9,P_cpi_10,...,GB_gscpi,F_fb_1,F_fb_2,F_fb_3,F_fb_4,F_fb_5,F_fb_6,T_nsi,T_usepu,T_uscepu
2000-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.20,71.91
2000-02-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.41,NaN,NaN,NaN,NaN,NaN,-1059.0,NaN,77.60,50.35
2000-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.28,NaN,NaN,NaN,NaN,NaN,5601.0,NaN,77.55,88.18
2000-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.14,NaN,NaN,NaN,NaN,NaN,1178.0,NaN,76.06,82.57
2000-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.29,NaN,NaN,NaN,NaN,NaN,3540.0,NaN,97.99,245.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-30,2.410939,3.696935,2.533800,1.160249,1.577473,1.790515,3.817223,1.519673,1.397693,2.957950,...,-0.34,4.758274,5.826992,5.487922,5.165472,9.988206,-23813.0,109.38,117.83,66.37
2024-07-31,2.551891,3.550867,2.469480,1.174286,2.182287,1.741362,5.083968,1.732603,1.633154,2.825566,...,-0.04,3.668043,4.692569,4.393595,3.800365,8.591109,23627.0,106.66,118.28,161.21
2024-08-31,2.012825,2.020437,2.441303,1.833776,1.837322,1.749428,1.806702,1.316354,1.849533,2.707935,...,0.20,NaN,NaN,NaN,NaN,NaN,NaN,99.47,114.18,224.67
2024-09-30,1.595038,1.828678,2.441303,1.801851,1.348768,1.922672,-1.204308,1.136323,1.914750,2.561024,...,0.13,NaN,NaN,NaN,NaN,NaN,NaN,98.84,117.88,85.75


### 오브젝트 스토리지(s3)에 생성된 데이터셋 저장

In [11]:
#s3_repo_path = f"s3://newtech/public/inf_nowcasting/input/mdata/{vintage_date}.xlsx"

#DF.to_excel(s3_repo_path, sheet_name="data")
#DF.to_csv(f"{s3_repo_path}/mdata/{vintage_date}.csv")
#DF_raw.to_csv(f"{s3_repo_path}/mdata/{vintage_date}_raw.csv")